In [153]:
import base64
import imageio
import IPython
import matplotlib.pyplot as plt
import os
import reverb
import tempfile
import PIL.Image

import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

import pandas_ta as ta

from tf_agents.environments import suite_gym
import tensorflow as tf

from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.trajectories import time_step as ts
from tf_agents.specs import array_spec
from tf_agents.environments import utils

from tf_agents.agents.ddpg import critic_network
from tf_agents.agents.sac import sac_agent
from tf_agents.agents.sac import tanh_normal_projection_network
from tf_agents.metrics import py_metrics
from tf_agents.networks import actor_distribution_network
from tf_agents.policies import greedy_policy
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_py_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.networks import Sequential
from tf_agents.train import actor
from tf_agents.train import learner
from tf_agents.train import triggers
from tf_agents.train.utils import spec_utils
from tf_agents.train.utils import strategy_utils
from tf_agents.train.utils import train_utils

import math

In [154]:
def get_relative_change(df, column_name):
    relative_changes = []
    for i in range(len(df)):
        if i == 0:
            relative_changes.append(0)  # First element has no previous value
        else:
            relative_change = (df.iloc[i] - df.iloc[i-1]) / df.iloc[i-1]
            relative_changes.append(relative_change)
    return pd.DataFrame({column_name: relative_changes})


In [162]:
df_csv = pd.read_csv('/tmp/eurusd_hour.csv')
pd_close = df_csv['BC']
bid_close = pd_close.to_numpy()


change = get_relative_change(pd_close,"Change")
rsi = ta.rsi(pd_close,14,scalar=1)
stdev = ta.stdev(pd_close,14)
macd = ta.macd(pd_close,14)


df = pd.concat([change,rsi,stdev,macd], axis=1)
df = df.dropna(axis=0)
df = df.reset_index()

change = df.pop('Change')
df.pop('index')

indicators = df.to_numpy()
change = change.to_numpy()
change_dev = np.std(change)
np.set_printoptions(suppress=True)
print(change_dev)

0.0012178838054657292


In [163]:
commission_rate = .005
default_val = 10
max_risk = .05
max_ep_len = 5000

In [164]:
class CardGameEnv(py_environment.PyEnvironment):
    def __init__(self):
        self._action_spec = array_spec.BoundedArraySpec(
            shape=(1,), dtype=np.float32, minimum=[-1], maximum=[1], name='action')
        self._observation_spec = array_spec.BoundedArraySpec(
            shape=(6,), dtype=np.float32, minimum=-10, maximum=10, name='observation')
        
       
        self._state = np.append(indicators[0],(0))
        self._episode_ended = False
        self._step_count = 1
        self._net_profit = 0
        self._prev_action = 0
        self._total_pen = 0

        
    def action_spec(self):
        return self._action_spec

    def observation_spec(self):
        return self._observation_spec
    
    

    def _reset(self):  
        
        self._state = np.append(indicators[0],(0))
        self._episode_ended = False
        self._step_count = 1
        self._net_profit = 0
        self._prev_action = 0
        self._total_pen = 0


        
        return ts.restart(np.array(self._state, dtype=np.float32))

    
    
    def _step(self, action):
        if self._episode_ended:
            return self.reset()          
        
        action = action[0]
        
        
        
        cur_val = (action*default_val) * change[self._step_count]
        
        cur_prec = (cur_val/default_val)
        
        self._net_profit += cur_val          
        
        discount_pen = 2 if cur_prec > 0 else 1 

        move_penalty = (abs(_prev_action - action)/discount_pen)/10
        
        self._total_pen -= move_penalty
        
        self._prev_action = action  
        
        #update state
        self._state = np.append(indicators[self._step_count],(action))
        
        #end episode if max steps or invalid_action
        if self._step_count >= max_ep_len:
            self._episode_ended = True
            
        self._step_count += 1

        if self._episode_ended:
            reward = self._total_pen   
            return ts.termination(np.array(self._state, dtype=np.float32), reward=reward)
        else:
            reward = cur_prec
            return ts.transition(np.array(self._state, dtype=np.float32), reward=reward,discount=.5)

In [165]:
environment = CardGameEnv()
utils.validate_py_environment(environment, episodes=10)